# **NLP Assignment 3**

- Import the necessary libraries!

In [0]:
import re
from string import digits
import string
import itertools
import copy

- Please skip this step. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **Load and clean the data**
- We read the data file and strip the text of leading and trailing spaces, split lines according \n in a list.

In [0]:
def readFile(path):
  file = open(path, mode='rt', encoding='utf-8')
  text = file.read()
  file.close()
  text = text.strip().split("\n")
  return text

- The data is preprocessed by making the words lower case, removing digits and removing punctuation except for the apostrophe (') to retain any possessive nouns. 
- The words are tokenized by white spaces.
- The list of words are stored in a list of sentences and returned.
- A vocabulary of unique words pertaining to the corpus is returned.

In [0]:
def cleanData(data):
  cleandata=[]
  for line in data:
    # convert to lower case
    line = line.lower()
    remove_digits = str.maketrans('', '', digits)
    line = line.translate(remove_digits)
    #remove punctuation except for apostrophe
    line = re.sub('[!#?,.:";)(]', '', line)
    # tokenize on white space
    line = line.split()
    cleandata.append(line)
  flat_list = [item for sublist in cleandata for item in sublist]
  vocab= set(flat_list)
  return cleandata, vocab
  

- The unzipped data is stored in the same folder where this notebook has been kept.


In [0]:
fr_path="/content/drive/My Drive/NLP Assignment 3/fr-en.fr"
fr = readFile(fr_path)
clean_fr, vocab_fr= cleanData(fr)

In [0]:
en_path="/content/drive/My Drive/NLP Assignment 3/fr-en.en"
en = readFile(en_path)
clean_en, vocab_en= cleanData(en)

## Problem 1

- Word translation probabilities are stored in a dictionary using the dictionary data structure. 
- The probabilities are initialized uniformly as a unit fraction of the French Vocabulary

In [0]:
def trans_prob():
  dictionary={}
  count = {}
  total = {}
  for fr in vocab_fr:
    total[fr]=0
    for eng in vocab_en:
      count[(eng, fr)] = 0
      dictionary[(eng, fr)]=1/(len(vocab_fr))
  return total, count, dictionary

## Problem 2

- The IBM model 1 has been implemented as per the pseudocode.
- The input is taken as a set of sentence pairs.
- The previously uniformly distributed translational probabilities are used here.
- Count and total dictionary is initialized to 0.
- Normalization has been computed for all sentence pairs.
- Counts are collected and probabilities are estimated.
- Before checking for convergence, the probabilities are rounded off to 3 decimal places.
- This loop converges when the translational probability table stops changing which is checked in the while loop. SInce convergence takes time, I have placed another condition which converges for number of iterations specified. It is 15 here

In [0]:
def model(total, count, dictionary, iterations):
  c=0
  old_t={}
  # new_dic=copy.deepcopy(dictionary)
  while(dictionary!=old_t and c<iterations):
    print("Loop started: ",c)
    old_t=copy.deepcopy(dictionary)
    for i in range(len(clean_en)): #Sent pairs
      for eng in clean_en[i]:
        add=0
        prod=1
        sum_c=0
        for fr in clean_fr[i]:
          add+=dictionary[(eng,fr)]
        for fr in clean_fr[i]:
          count[(eng,fr)]+=dictionary[(eng,fr)]/add
          total[fr]+=dictionary[(eng,fr)]/add

    for f in vocab_fr:
      for e in vocab_en:
        dictionary[(e,f)]=round(count[(e,f)]/total[f],3)
    c+=1
  return dictionary


- Gives out initial uniform word translation probability

In [0]:
total, count, dictionary = trans_prob()

In [12]:
iterations=15
dictionary = model(total, count, dictionary, iterations)

Loop started:  0
Loop started:  1
Loop started:  2
Loop started:  3
Loop started:  4
Loop started:  5
Loop started:  6
Loop started:  7
Loop started:  8
Loop started:  9
Loop started:  10
Loop started:  11
Loop started:  12
Loop started:  13
Loop started:  14


- After the calculation of final translational probabilities, this dictionary is used to retrieve english aligned sentence for a given french sentence.
- For a given word pair which includes the French word, the corresponding high valued english word is outputted.

In [0]:
def alignments(clean_fr, dictionary):
  eng_sent=[]
  for sent in clean_fr:
    eng=[]
    for word in sent:
      maxVal = 0
      maxKey = ()
      for key, value in dictionary.items():
      # This will check if testValue matches any item in tuple.
        if word == key[1]:
          if maxVal < value:
            maxVal = value
            maxKey = key
      eng.append(maxKey[0])
    eng_sent.append(eng)
  return eng_sent


In [0]:
alignments = alignments(clean_fr[:20], dictionary)

- Print English alignments for 20 French sentences.

In [20]:
for i in range(20):
  print(clean_fr[:20][i])
  print(alignments[i])
  print()
  print()

['reprise', 'de', 'la', 'session']
['session', 'the', 'the', 'part-session']


['je', 'déclare', 'reprise', 'la', 'session', 'du', 'parlement', 'européen', 'qui', 'avait', 'été', 'interrompue', 'le', 'vendredi', 'décembre', 'dernier', 'et', 'je', 'vous', 'renouvelle', 'tous', 'mes', 'vux', 'en', 'espérant', 'que', 'vous', 'avez', 'passé', 'de', 'bonnes', 'vacances']
['i', 'enjoyed', 'session', 'the', 'part-session', 'the', 'parliament', 'european', 'which', 'in', 'been', 'enjoyed', 'the', 'friday', 'december', 'like', 'and', 'i', 'you', 'enjoyed', 'all', 'my', 'happy', 'the', 'enjoyed', 'that', 'you', 'you', 'past', 'the', 'these', 'enjoyed']


['comme', 'vous', 'avez', 'pu', 'le', 'constater', 'le', 'grand', 'bogue', 'de', "l'", 'an', 'ne', "s'", 'est', 'pas', 'produit', 'en', 'revanche', 'les', 'citoyens', "d'", 'un', 'certain', 'nombre', 'de', 'nos', 'pays', 'ont', 'été', 'victimes', 'de', 'catastrophes', 'naturelles', 'qui', 'ont', 'vraiment', 'été', 'terribles']
['as', 'you', 'you